## Ejercicio 2


### Importamos librerias necesarias


In [193]:
import pandas as pd


### Cargamos los archivos de excel, estos se encuentran en data/ejercicio2


In [194]:
df_ejercicio2 = pd.read_excel(r"data/ejercicio2/segundoejercicio.xlsx")
df_butacas = pd.read_excel(r"data/ejercicio2/butacas.xlsx")


### Archivo segundoejercicio.xlsx


In [195]:
# Eliminamos columnas y filas no neceserias
df_ejercicio2.drop(df_ejercicio2.index[range(6)], axis=0, inplace=True)

# Elimina columnas que no sean necesarias
df_ejercicio2.drop(df_ejercicio2.columns[[3, 4, 5, 8]], axis=1, inplace=True)


# Renombramos columnas
df_ejercicio2.columns = ["Cinema", "Screen", "Film", "Number of Sessions", "Admits"]


### Limpieza de datos


In [196]:
def split_cell(cell, separador, posicion):
    """
    Funcion que separa un string segun el separador
    y devuelve la posicion indicada
    """
    return cell.split(separador)[posicion]


# Elimina filas que contengan el valor Total
df_ejercicio2.drop(df_ejercicio2[df_ejercicio2["Film"] == "Total"].index, inplace=True)
df_ejercicio2.drop(
    df_ejercicio2[df_ejercicio2["Screen"] == "Total"].index, inplace=True
)
df_ejercicio2.drop(
    df_ejercicio2[df_ejercicio2["Cinema"] == "Total"].index, inplace=True
)

df_ejercicio2 = df_ejercicio2.iloc[:-1]


# Llenamos celdas vacias con el valor de la celda anterior
df_ejercicio2.fillna(method="ffill", inplace=True)

# A las columnas cinema y Film le aplicamos la funcion split_cell
df_ejercicio2["Cinema"] = df_ejercicio2["Cinema"].apply(split_cell, args=(" ", 1))
df_ejercicio2["Film"] = df_ejercicio2["Film"].apply(split_cell, args=(" (", 0))

# Nuevo dataframe para los datos limpios
df_ejercicio2_filtrado = pd.DataFrame()
df_ejercicio2_filtrado["Cinema"] = df_ejercicio2["Cinema"]
df_ejercicio2_filtrado["Screen"] = df_ejercicio2["Screen"]
df_ejercicio2_filtrado["Film"] = df_ejercicio2["Film"]

# Eliminamos duplicados
df_ejercicio2_filtrado.drop_duplicates(inplace=True)

# Realizamos una sumatoria con las columnas Number of session y Admits
df_ejercicio2_filtrado["Number of session"] = df_ejercicio2.groupby(
    ["Cinema", "Screen", "Film"]
)["Number of Sessions"].transform("sum")
df_ejercicio2_filtrado["Admits"] = df_ejercicio2.groupby(["Cinema", "Screen", "Film"])[
    "Admits"
].transform("sum")

df_ejercicio2_filtrado.to_csv(r"data/ejercicio2/ejercicio2.csv", index=False)


### Obtener el total de butacas disponibles para cada pelicula


In [197]:
df_ejercicio2_filtrado['Cinema'] = pd.to_numeric(df_ejercicio2_filtrado['Cinema'])
df_ejercicio2_filtrado['Screen'] = pd.to_numeric(df_ejercicio2_filtrado['Screen'])

df_butacas['Theatre'] = pd.to_numeric(df_butacas['Theatre'])
df_butacas['Screen'] = pd.to_numeric(df_butacas['Screen'])

df_merge = pd.merge(
    df_ejercicio2_filtrado,
    df_butacas,
    left_on=["Cinema", "Screen"],
    right_on=["Theatre", "Screen"],
    how="left",
    copy=False,
)

# Calculamos la ocupacion de las butacas multiplicando el numero de butacas por el numero de sesiones
df_merge["Maxbutacas"] = df_merge["Number of session"] * df_merge["Seats"]

### Obtener el porcentaje de ocupacion de cada pelicula

In [198]:
df_porcentaje = pd.DataFrame()

# En un dataframe copiamos la columna Film
df_porcentaje["Film"] = df_merge["Film"]

# Eliminamos duplicados
df_porcentaje.drop_duplicates(inplace=True)

# Realizamo una sumatoria con las columnas Maxbutacas y admits
df_porcentaje["Maxbutacas"] = df_merge.groupby(["Film"])["Maxbutacas"].transform("sum")
df_porcentaje["Admits"] = df_merge.groupby(["Film"])["Admits"].transform("sum")

# Divide la columna Admits entre Maxbutacas y redondea a 2 decimales
# pd: no se dejo usar f strings :c
df_porcentaje["Porcentaje"] = (round(df_porcentaje["Admits"] / df_porcentaje["Maxbutacas"] * 100, 0))
df_porcentaje["Porcentaje"] = df_porcentaje["Porcentaje"].apply(lambda x: f"{x}%")

# exportamos el dataframe a xlsx
df_porcentaje.to_excel(r"data/ejercicio2/porcentaje.xlsx", index=False, header=True)